# Buck Converter Simulation

This notebook demonstrates how to simulate a synchronous buck DC-DC converter with Pulsim.

## Contents
1. Circuit Description
2. Building the Circuit
3. Simulation and Analysis
4. Ripple Analysis
5. Efficiency Calculation

## 1. Circuit Description

A synchronous buck converter steps down voltage from Vin to Vout.

```
        S_high
Vin ──┬──/  /──┬──LLLL──┬── Vout
      │        │   L    │
      │   S_low│        ═ C
      │   ─┴─  │        │
GND ──┴────────┴────────┴── GND
```

**Design Parameters:**
- Input voltage: 12V
- Output voltage: 5V (D ≈ 0.417)
- Switching frequency: 100 kHz
- Output current: 2A
- Inductor: 100 µH
- Capacitor: 100 µF

In [ ]:
import pulsim
import numpy as np
import matplotlib.pyplot as plt

# Design parameters
VIN = 12.0       # Input voltage (V)
VOUT = 5.0       # Target output voltage (V)
FSW = 100e3      # Switching frequency (Hz)
L = 100e-6       # Inductance (H)
C = 100e-6       # Capacitance (F)
R_LOAD = 2.5     # Load resistance (Ω) -> 2A @ 5V

# Calculate duty cycle
D = VOUT / VIN
print(f"Duty cycle: {D:.3f} ({D*100:.1f}%)")
print(f"Switching period: {1/FSW*1e6:.1f} µs")

## 2. Building the Circuit

In [ ]:
# Create circuit
circuit = pulsim.Circuit("Synchronous Buck Converter")

# Input voltage source
circuit.add_voltage_source("Vin", "vin", "0", VIN)

# High-side switch (controlled by PWM)
circuit.add_switch("S_high", "vin", "sw", 
                   control="PWM_high",
                   ron=0.01)  # 10mΩ on-resistance

# Low-side switch (complementary PWM)
circuit.add_switch("S_low", "sw", "0",
                   control="PWM_low",
                   ron=0.01)

# Output LC filter
circuit.add_inductor("L1", "sw", "vout", L, ic=0)
circuit.add_capacitor("C1", "vout", "0", C, ic=0)

# Load resistor
circuit.add_resistor("R_load", "vout", "0", R_LOAD)

# PWM sources
circuit.add_pwm("PWM_high", frequency=FSW, duty_cycle=D)
circuit.add_pwm("PWM_low", frequency=FSW, duty_cycle=D, 
                inverted=True, dead_time=100e-9)  # 100ns dead time

print("Circuit created successfully!")

## 3. Simulation and Analysis

In [ ]:
# Configure simulation
# Simulate for 1ms (100 switching cycles)
options = pulsim.SimulationOptions(
    stop_time=1e-3,
    timestep=10e-9,  # 10ns timestep for accurate switching
    abstol=1e-12,
    reltol=1e-3
)

# Run simulation
print("Running simulation...")
result = pulsim.simulate(circuit, options)
print(f"Simulation completed in {result.total_steps} steps")

In [ ]:
# Plot overview
fig, axes = plt.subplots(4, 1, figsize=(12, 10), sharex=True)

time_ms = result.time * 1e3

# Switch node voltage
axes[0].plot(time_ms, result.voltages["sw"], 'b-', linewidth=0.5)
axes[0].set_ylabel('V_sw (V)')
axes[0].set_title('Switch Node Voltage')
axes[0].grid(True)

# Output voltage
axes[1].plot(time_ms, result.voltages["vout"], 'r-', linewidth=1)
axes[1].axhline(y=VOUT, color='g', linestyle='--', label=f'Target: {VOUT}V')
axes[1].set_ylabel('V_out (V)')
axes[1].set_title('Output Voltage')
axes[1].legend()
axes[1].grid(True)

# Inductor current
axes[2].plot(time_ms, result.currents["L1"], 'g-', linewidth=0.5)
axes[2].set_ylabel('I_L (A)')
axes[2].set_title('Inductor Current')
axes[2].grid(True)

# Input current
axes[3].plot(time_ms, -result.currents["Vin"], 'm-', linewidth=0.5)
axes[3].set_ylabel('I_in (A)')
axes[3].set_xlabel('Time (ms)')
axes[3].set_title('Input Current')
axes[3].grid(True)

plt.tight_layout()
plt.show()

## 4. Ripple Analysis

Let's analyze the steady-state ripple after the converter has settled.

In [ ]:
# Extract last 10 switching cycles (steady state)
T_sw = 1 / FSW
t_start = result.time[-1] - 10 * T_sw

mask = result.time >= t_start
t_ss = result.time[mask]
v_out_ss = result.voltages["vout"][mask]
i_L_ss = result.currents["L1"][mask]

# Calculate ripple
v_ripple_pp = np.max(v_out_ss) - np.min(v_out_ss)
i_ripple_pp = np.max(i_L_ss) - np.min(i_L_ss)
v_out_avg = np.mean(v_out_ss)
i_L_avg = np.mean(i_L_ss)

print("Steady-State Analysis:")
print(f"  Output voltage: {v_out_avg:.3f} V")
print(f"  Voltage ripple: {v_ripple_pp*1e3:.2f} mV ({v_ripple_pp/v_out_avg*100:.2f}%)")
print(f"  Inductor current: {i_L_avg:.3f} A")
print(f"  Current ripple: {i_ripple_pp*1e3:.1f} mA ({i_ripple_pp/i_L_avg*100:.1f}%)")

In [ ]:
# Theoretical ripple calculations
# Current ripple: ΔI_L = (Vin - Vout) * D * T / L
delta_IL_theory = (VIN - VOUT) * D / (L * FSW)

# Voltage ripple: ΔV_out ≈ ΔI_L / (8 * f_sw * C)
delta_V_theory = delta_IL_theory / (8 * FSW * C)

print("\nTheoretical vs Simulated:")
print(f"  Current ripple: {delta_IL_theory*1e3:.1f} mA (theory) vs {i_ripple_pp*1e3:.1f} mA (sim)")
print(f"  Voltage ripple: {delta_V_theory*1e3:.2f} mV (theory) vs {v_ripple_pp*1e3:.2f} mV (sim)")

In [ ]:
# Zoom in on steady-state waveforms
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# Show 3 switching cycles
t_zoom_start = t_start + 5 * T_sw
t_zoom_end = t_start + 8 * T_sw
zoom_mask = (result.time >= t_zoom_start) & (result.time <= t_zoom_end)

t_zoom = (result.time[zoom_mask] - t_zoom_start) * 1e6  # µs

axes[0].plot(t_zoom, result.voltages["vout"][zoom_mask], 'r-', linewidth=1.5)
axes[0].set_ylabel('V_out (V)')
axes[0].set_title(f'Output Voltage Ripple: {v_ripple_pp*1e3:.2f} mV p-p')
axes[0].grid(True)

axes[1].plot(t_zoom, result.currents["L1"][zoom_mask], 'g-', linewidth=1.5)
axes[1].set_ylabel('I_L (A)')
axes[1].set_xlabel('Time (µs)')
axes[1].set_title(f'Inductor Current Ripple: {i_ripple_pp*1e3:.1f} mA p-p')
axes[1].grid(True)

plt.tight_layout()
plt.show()

## 5. Efficiency Calculation

In [ ]:
# Calculate power (using steady-state data)
v_in_ss = result.voltages["vin"][mask]
i_in_ss = -result.currents["Vin"][mask]  # Negative because current flows out of source

# Average input power
P_in = np.mean(v_in_ss * i_in_ss)

# Output power
P_out = np.mean(v_out_ss ** 2) / R_LOAD

# Losses
P_loss = P_in - P_out

# Efficiency
efficiency = P_out / P_in * 100

print("Power Analysis (Steady State):")
print(f"  Input power:  {P_in:.3f} W")
print(f"  Output power: {P_out:.3f} W")
print(f"  Losses:       {P_loss*1e3:.1f} mW")
print(f"  Efficiency:   {efficiency:.2f}%")

In [ ]:
# If loss breakdown is available
if hasattr(result, 'losses'):
    losses = result.losses
    print("\nLoss Breakdown:")
    print(f"  Conduction: {losses.conduction_energy * FSW * 1e3:.2f} mW")
    print(f"  Switching:  {losses.switching_energy * FSW * 1e3:.2f} mW")
    
    for device, loss in losses.by_device.items():
        print(f"  {device}: {loss * FSW * 1e3:.2f} mW")

## Summary

In this notebook, we:
- Built a synchronous buck converter model
- Simulated startup and steady-state operation
- Analyzed output voltage and inductor current ripple
- Compared simulation results with theoretical predictions
- Calculated power efficiency

The simulation shows good agreement with theoretical values, demonstrating Pulsim's accuracy for power electronics simulation.

**Next:** [Thermal Modeling Tutorial](03_thermal_modeling.ipynb)